<h1>Predictive Maintenance of Water Pump in Africa</h1>

This project is about using data analysis to predict when a water pump will be non-functional. The Dataset comes from the drivendata.org. <br>
To start this project we will like to import the data into our notebook and begin some basic analysis on it.

In [2]:
import pandas as pd
import numpy as np

df_values = pd.read_csv("Training_Set_Values.csv")
df_labels = pd.read_csv("Training_set_labels.csv")

df_values.shape, df_labels.shape


((59400, 40), (59400, 2))

As can be seen from the result above, there are 40 features and 59400 observations in the dataset. <br>
The first feature is the ID Number of the pump, which we will ignore.<br> 
Out of the other 39 features, we will drop some features manually.<br>
Possible reasons for drop are:<br>
* Duplicates/Redundant Column
* Feature assumed to have little to no correlation



In [3]:
dropped_df = df_values.drop(columns=["longitude", "latitude", "wpt_name", "num_private",
                              "recorded_by", "permit", "payment", "payment_type", 
                              "waterpoint_type_group", "basin", "subvillage", 
                              "region", "source", "extraction_type_group", 
                              "extraction_type_class", "district_code", "quantity_group", "lga", "ward"])

#Include Status into the DF so that any mutations we do to the df is done to the Status Series as well.
dropped_df["Status"] = df_labels["status_group"]
dropped_df.shape

(59400, 22)

After dropping some features, we are now left with 20 features. However not all of the observations have complete data and thus we will have to do some preprocessing to the new df. Some of the possible step we will take for the preprocessing are: <br>
* Dropping Observations that are not able to be predicted and is considered important e.g. Population
* Predicting Values that may seem possible to be predicted
* Leaving the NaN Values as it is and use one hot encoder.<br>

Columns that have string values such as funder, installer, extraction type, etcwill be encoded using one hot encoder.<br> For these Columns, we will not drop rows that has NaN Values to avoid losing too much data<br>


In [4]:
#Drop rows with population = 0
dropped_df1= dropped_df[dropped_df["population"] != 0]
dropped_df1=pd.get_dummies(dropped_df1, columns=["funder", "installer", "public_meeting", 
                                    "scheme_management", "scheme_name", "extraction_type", 
                                    "management", "management_group", "water_quality", 
                                    "quality_group", "quantity", "source_type", 
                                    "source_class", "waterpoint_type"])


After Using the pandas OneHotEncoder Method, we now have a dataset that looks ready for fitting into the model.<br>
However, one of the feature that can be a problem is the date_recorded as it is not in a suitable data form for fitting.<br>
The optimal way to work with a date time feature is to convert it to a np.datetime format which has lot of built-in methods and function to it.<br>


In [5]:
#Used to turn off the warning that occurs because of chained assignment
pd.set_option('mode.chained_assignment', None)

dropped_df1["date_recorded"] =pd.to_datetime(dropped_df1["date_recorded"])
dropped_df1.dtypes["date_recorded"]

dtype('<M8[ns]')

Now that the date_recorded column is in np.datetime format, lets separate the year-month-date into separate columns that will <br>
fitted into the classifier that we will be creating later. As the value of date may not be as important as month or year,<br>
We will not be using the date value in this case.

In [6]:
dropped_df1["year"] = dropped_df1["date_recorded"].dt.year
dropped_df1["month"] = dropped_df1["date_recorded"].dt.month
dropped_df1.shape

(38019, 5044)

Now the dataframe is almost ready. We just need to drop the 3 unnecessary columns left which are:
* id
* date_recorded
* Status (needs to be stored in a series)

In [7]:
target= dropped_df1["Status"]
data=dropped_df1.drop(columns=["date_recorded", "id", "Status"])

Now that all the data are ready to be fitted into the model,<br>
We can begin thinking about the model that we will want to use.<br>
<br>
We will be starting by splitting the dataframe into train set and test set<br>

In [8]:
from sklearn.model_selection import train_test_split

x_train,x_test, y_train, y_test = train_test_split(data, target, test_size=0.2, random_state=42)

With both training set and test set ready, we now have to make a decision on which model and hyperparameter to use.<br>
We will use 3 types of model with a small parameter grid to check each model accuracy.<br>

<h1>KNeighborsClassifier</h1>


In [9]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

KNN_pipe= Pipeline([('scaler', StandardScaler()), ('clf', KNeighborsClassifier())])
KNN_grid= {
    "clf__n_neighbors" : np.arange(5,15,2)
}
KNN_clf = GridSearchCV(KNN_pipe, KNN_grid, n_jobs=3, cv=3, scoring="f1_micro")
KNN_clf.fit(x_train,y_train)

print(classification_report(y_test, KNN_clf.predict(x_test)))


                         precision    recall  f1-score   support

             functional       0.78      0.86      0.82      4256
functional needs repair       0.45      0.26      0.33       498
         non functional       0.78      0.71      0.74      2850

               accuracy                           0.76      7604
              macro avg       0.67      0.61      0.63      7604
           weighted avg       0.76      0.76      0.76      7604



In [10]:
KNN_clf.best_score_

0.7424626033765563

In [11]:
KNN_clf.best_params_

{'clf__n_neighbors': 5}

In [12]:
KNN_clf.cv_results_

{'mean_fit_time': array([5.44389399, 4.30897204, 4.4329985 , 4.42566371, 4.13626528]),
 'std_fit_time': array([0.06586458, 0.15512784, 0.11398189, 0.08609851, 0.13118932]),
 'mean_score_time': array([43.88607184, 44.30823533, 42.57977978, 41.53908308, 40.82308348]),
 'std_score_time': array([0.13875492, 0.10594938, 0.08656967, 0.14984577, 0.1508552 ]),
 'param_clf__n_neighbors': masked_array(data=[5, 7, 9, 11, 13],
              mask=[False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'clf__n_neighbors': 5},
  {'clf__n_neighbors': 7},
  {'clf__n_neighbors': 9},
  {'clf__n_neighbors': 11},
  {'clf__n_neighbors': 13}],
 'split0_test_score': array([0.74238091, 0.73695631, 0.73153171, 0.7240359 , 0.72275372]),
 'split1_test_score': array([0.73949497, 0.73426711, 0.73051884, 0.73120931, 0.72440324]),
 'split2_test_score': array([0.74551194, 0.74353916, 0.7388045 , 0.72992701, 0.72834879]),
 'mean_test_score': array([0.7424626 , 0.73825419, 0

<h1>LogisticRegression</h1>

Next LogisticRegression classifier. The LogisticRegression Classifier is basically a classifier that uses a linear boundary.<br>
The important things to note here is since there are more than 2 possible result (non-binary classifier), the classifier will use a one vs rest implementation 
instead of a 0.5 probability threshold<br>


In [13]:
from sklearn.linear_model import LogisticRegression

Logres_pipe= Pipeline([('scaler', StandardScaler()), ('clf', LogisticRegression())])
Logres_grid= {
    "clf__C" : np.logspace(0.1,1000, num=5)
}
Logres_clf = GridSearchCV(Logres_pipe, Logres_grid, n_jobs=3, cv=3, scoring="f1_micro")
Logres_clf.fit(x_train,y_train)

print(classification_report(y_test, Logres_clf.predict(x_test)))

C:\Users\johan\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\core\function_base.py:277: RuntimeWarning: overflow encountered in power
  return _nx.power(base, y)
C:\Users\johan\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


                         precision    recall  f1-score   support

             functional       0.79      0.88      0.83      4256
functional needs repair       0.49      0.23      0.32       498
         non functional       0.80      0.73      0.76      2850

               accuracy                           0.78      7604
              macro avg       0.69      0.61      0.64      7604
           weighted avg       0.77      0.78      0.77      7604



In [14]:
print(classification_report(y_test, Logres_clf.predict(x_test)))

                         precision    recall  f1-score   support

             functional       0.79      0.88      0.83      4256
functional needs repair       0.49      0.23      0.32       498
         non functional       0.80      0.73      0.76      2850

               accuracy                           0.78      7604
              macro avg       0.69      0.61      0.64      7604
           weighted avg       0.77      0.78      0.77      7604



In [15]:
Logres_clf.best_score_

0.7682394437188925

In [16]:
Logres_clf.best_params_

{'clf__C': 1.2589254117941673}

In [17]:
Logres_clf.cv_results_

{'mean_fit_time': array([79.88552841, 75.65994493, 75.31346472, 75.53257155, 75.55832132]),
 'std_fit_time': array([0.02131721, 0.90145538, 1.58451956, 1.7372809 , 1.46442059]),
 'mean_score_time': array([1.18653472, 1.35065285, 1.54172866, 1.47275599, 1.33677642]),
 'std_score_time': array([0.02015862, 0.59873186, 0.66701422, 0.73799537, 0.72158039]),
 'param_clf__C': masked_array(data=[1.2589254117941673, 1.1885022274369873e+250, inf, inf,
                    inf],
              mask=[False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'clf__C': 1.2589254117941673},
  {'clf__C': 1.1885022274369873e+250},
  {'clf__C': inf},
  {'clf__C': inf},
  {'clf__C': inf}],
 'split0_test_score': array([0.76555873, 0.7647697 , 0.7647697 , 0.7647697 , 0.7647697 ]),
 'split1_test_score': array([0.76790294, 0.76691655, 0.76691655, 0.76691655, 0.76691655]),
 'split2_test_score': array([0.77125666, 0.77066483, 0.77066483, 0.77066483, 0.77066483]),
 'mean

<h1>Decision Tree Classifier</h1>

In [18]:
from sklearn.tree import DecisionTreeClassifier

DTC_pipe= Pipeline([('scaler', StandardScaler()), ('clf', DecisionTreeClassifier())])
DTC_grid= {
    "clf__max_features" : ["auto", "sqrt", "log2"]
}
DTC_clf = GridSearchCV(DTC_pipe, DTC_grid, n_jobs=3, cv=3, scoring="f1_micro")
DTC_clf.fit(x_train,y_train)

print(classification_report(y_test, DTC_clf.predict(x_test)))

                         precision    recall  f1-score   support

             functional       0.80      0.79      0.80      4256
functional needs repair       0.33      0.37      0.35       498
         non functional       0.75      0.74      0.74      2850

               accuracy                           0.75      7604
              macro avg       0.63      0.63      0.63      7604
           weighted avg       0.75      0.75      0.75      7604



In [19]:
DTC_clf.best_score_


0.7365774055872922

In [20]:
DTC_clf.best_params_

{'clf__max_features': 'auto'}

In [21]:
DTC_clf.cv_results_

{'mean_fit_time': array([5.3961304 , 5.37050358, 5.24420102]),
 'std_fit_time': array([0.02585535, 0.01068068, 0.02581221]),
 'mean_score_time': array([1.01470327, 1.02189596, 1.05256923]),
 'std_score_time': array([0.00920362, 0.0102132 , 0.01357556]),
 'param_clf__max_features': masked_array(data=['auto', 'sqrt', 'log2'],
              mask=[False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'clf__max_features': 'auto'},
  {'clf__max_features': 'sqrt'},
  {'clf__max_features': 'log2'}],
 'split0_test_score': array([0.73478647, 0.74267679, 0.73064405]),
 'split1_test_score': array([0.73742355, 0.73584533, 0.72627737]),
 'split2_test_score': array([0.73752219, 0.72677057, 0.72617873]),
 'mean_test_score': array([0.73657741, 0.73509756, 0.72770005]),
 'std_test_score': array([0.00126702, 0.00651518, 0.00208211]),
 'rank_test_score': array([1, 2, 3])}

<h2> Data Visualization </h2>

First let us prepare some useful lists that will most probably be used multiple times in the future<br>

In [22]:
#Add all the classifier model to a list for future use
model_list= [KNN_clf,Logres_clf,DTC_clf]
model_name_list = ["kNearestNeighbors", "Logistic Regression", "Decision Tree Classifier"]
score_list=[]
label_list=["functional", "functional needs repair","non functional"]

for model in model_list:
    score_list.append(round(model.best_score_,4))

In [23]:
y_pred_knn=KNN_clf.predict(x_test)
y_pred_logres=Logres_clf.predict(x_test)
y_pred_dtc= DTC_clf.predict(x_test)

Now let us compare the scores of the classifier models side by side using a bar chart.<br>
We will be using the plotly module for visualization<br>

In [24]:
import plotly.express as px
import plotly.graph_objects as go

fig1= px.bar(y=model_name_list, x=score_list, hover_name=score_list)
fig1.update_layout(title_text="Classifier Score", title_x=0.5)
fig1.update_xaxes(title_text="Accuracy Score")
fig1.update_yaxes(title_text="Classification Model")

As can be seen, these are the accuracy score of each model. We would then also visualize a confusionmatrix<br>
which will allows us to compare prediction probabilities.

In [25]:
from sklearn.metrics import confusion_matrix

cf1=confusion_matrix(y_test, y_pred_knn, normalize="true", labels=label_list)
cf2=confusion_matrix(y_test, y_pred_logres, normalize="true", labels=label_list)
cf3=confusion_matrix(y_test, y_pred_dtc, normalize="true", labels=label_list)

In [26]:
fig2 =go.Figure()
for cf in [cf1,cf2,cf3]:
    fig2.add_trace(go.Heatmap(
        x=label_list,y=label_list,z=cf, hoverinfo="text", hovertext=np.round(cf,5)
    ))
dropdown_buttons = [  
    {'label': 'KNN', 'method': 'update','args': [{'visible': [True, False, False]}, {'title': 'KNearestNeighbors classification Matrix'}]},  
    {'label': 'LogisticRegression', 'method': 'update','args': [{'visible': [False, True, False]}, {'title': 'LogisticRegression classification Matrix'}]},  
    {'label': "DecisionTreeClassifier", 'method': "update",'args': [{"visible": [False, False, True]}, {'title': 'DecisionTreeClassifier classification Matrix'}]}
    ]

fig2.update_xaxes(title_text="Predicted Label")
fig2.update_yaxes(title_text="True Label")
fig2.update_layout(title_text="KNearestNeighbors classification Matrix")
fig2.data[1].visible=False
fig2.data[2].visible=False
fig2.update_layout(updatemenus=[{'type': "dropdown",'x': 1.25,'y': 0.5,'showactive': True,'active': 0,'buttons': dropdown_buttons}], title_x=0.5)
fig2.show()